In [ ]:
## Importing the Packages 
import numpy as np
import cv2

from keras.models import Sequential

from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D
from keras.optimizers import Adam
from keras.layers import MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
#Build the convolution network architecture:

from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense

emotion_model = Sequential()

emotion_model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(48, 48, 1)))
emotion_model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Dropout(0.25))

emotion_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Dropout(0.25))

emotion_model.add(Flatten())
emotion_model.add(Dense(1024, activation='relu'))
emotion_model.add(Dropout(0.5))
emotion_model.add(Dense(7, activation='softmax'))


In [ ]:
emotion_model.load_weights('model.h5')

In [ ]:
import os
# Load the Haar cascade classifier for face detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Emotion dictionary
emotion_dict = {
    0: "Angry",
    1: "Disgusted",
    2: "Fearful",
    3: "Happy",
    4: "Neutral",
    5: "Sad",
    6: "Surprised"
}

# Path to the directory containing emoji images
emoji_dir = "E:\Face Emotion Dataset\emoji_folder"

# Check if the emoji directory exists
if not os.path.exists(emoji_dir):
    print(f"Error: Emoji directory '{emoji_dir}' not found.")
    exit(1)

# Emoji dictionary
emoji_dict = {}
for i in range(len(emotion_dict)):
    emoji_path = os.path.join(emoji_dir, f"{i}.jpg")
    if os.path.exists(emoji_path):
        emoji_dict[i] = emoji_path
    else:
        print(f"Error: Emoji image '{emoji_path}' not found.")
        exit(1)

# Function to detect and process faces
def detect_faces(frame):
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray_frame, scaleFactor=1.3, minNeighbors=5)
    
    for (x, y, w, h) in faces:
        # Preprocess the face region
        face_roi = gray_frame[y:y+h, x:x+w]
        resized_roi = cv2.resize(face_roi, (48, 48))
        normalized_roi = resized_roi / 255.0
        reshaped_roi = normalized_roi.reshape(1, 48, 48, 1)
        
        # Make prediction using the emotion model
        prediction = emotion_model.predict(reshaped_roi)
        max_index = int(prediction.argmax())
        emotion_label = emotion_dict[max_index]  # Get the emotion label
        
        # Draw bounding box and text label on the frame
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
        label = f"Emotion: {emotion_label}"
        cv2.putText(frame, label, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2, cv2.LINE_AA)
        
        # Display the corresponding emoji in a new window
        if max_index in emoji_dict:
            emoji_image = cv2.imread(emoji_dict[max_index])
            cv2.imshow('Emoji', emoji_image)
        else:
            print(f"Error: Emoji image not found for emotion index {max_index}")
    
    return frame

# Function to process video stream
def process_video_stream(url):
    cap = cv2.VideoCapture(url)
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        frame = cv2.resize(frame, (800, 600))
        
        processed_frame = detect_faces(frame)
        
        cv2.imshow('Video Stream', processed_frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    cap.release()
    cv2.destroyAllWindows()

if __name__ == '__main__':
    url = 'http://192.168.1.6:8080/video'
    process_video_stream(url)